# Import

In [1]:
import pandas as pd
import numpy as np
import os
import re
import emoji


import warnings
warnings.filterwarnings('ignore')

# txt to csv

In [2]:
df = pd.read_csv(f"../data/chat_ju.txt", delimiter='\t', encoding='utf-8')
df = df.to_csv(f"../data/chat_ju.csv", index=False)
df = pd.read_csv(f"../data/chat_ju.csv")
df

,"[11/1/21, 21:10:17] Ju: ‎Los mensajes y las llamadas están cifrados de extremo a extremo. Nadie fuera de este chat, ni siquiera WhatsApp, puede leerlos ni escucharlos."
0,"‎[11/1/21, 21:10:17] 🐥: ‎audio omitido"
1,"‎[12/1/21, 13:14:04] 🐥: ‎audio omitido"
2,"‎[12/1/21, 13:37:42] Ju: ‎audio omitido"
3,"[12/1/21, 14:22:35] 🐥: Que guai amooooor la of..."
4,"[12/1/21, 14:22:50] 🐥: Oiiii donali besis a la..."
...,...
18066,"[28/4/23, 15:09:53] 🐥: tinc encara 10 min sino..."
18067,"[28/4/23, 15:09:55] 🐥: com vlguis"
18068,"[28/4/23, 15:10:00] 🐥: uina h vas casa leni"
18069,"[28/4/23, 15:11:25] Ju: despres millor doncs amor"


# transform

## Setting columns

In [3]:
df.columns = ["col"]

#df.iloc[23]["col"].split(",")[0].replace("[","") 
df["Date"] = df["col"].str.split(",").str[0].str.replace("[", "")

#df.iloc[23]["col"].split(",")[1].split("]")[0].strip()
df["Time"] = df["col"].str.split(",").str[1].str.split("]").str[0].str.strip()

#df.iloc[22]["col"].split(":")[2].split("]")[1].strip()
df["Name"] = df["col"].str.split(":").str[2].str.split("]").str[1].str.strip()
df['Name'] = df['Name'].str.replace('🐥', 'Bego')

# df.iloc[0]["col"].split(":")[3].strip()
df["Message"] = df["col"].str.split(":").str[3].str.strip()

df.drop(columns=["col"], axis=1, inplace=True)
df.dropna(subset=['Message'], inplace=True) 

df

,Date,Time,Name,Message
0,‎11/1/21,21:10:17,Bego,‎audio omitido
1,‎12/1/21,13:14:04,Bego,‎audio omitido
2,‎12/1/21,13:37:42,Ju,‎audio omitido
3,12/1/21,14:22:35,Bego,Que guai amooooor la ofi no?
4,12/1/21,14:22:50,Bego,Oiiii donali besis a la cami amore
...,...,...,...,...
18066,28/4/23,15:09:53,Bego,tinc encara 10 min sino dpres
18067,28/4/23,15:09:55,Bego,com vlguis
18068,28/4/23,15:10:00,Bego,uina h vas casa leni
18069,28/4/23,15:11:25,Ju,despres millor doncs amor


In [4]:
pd.isnull(df).sum()

Date       0
Time       0
Name       0
Message    0
dtype: int64

## Emojis extraction

In [5]:
emoji_pattern = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U0001F900-\U0001F9FF"  # supplemental symbols and pictographs
    u"\U0001F1F2-\U0001F1F4"  # country flags
    u"\U0001F1E6-\U0001F1FF"  # regional indicator symbols
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U00002702-\U000027B0"  # dingbats
    u"\U000024C2-\U0001F251" 
    "]+", flags=re.UNICODE)

In [6]:
df['Emojis'] = df['Message'].str.findall(emoji_pattern)

In [7]:
#df = df[df['Emojis'].apply(lambda x: len(x) > 0)]
#df.drop(columns=["Message"], axis=1, inplace=True)
#df = df.reset_index(drop=True)
df

,Date,Time,Name,Message,Emojis
0,‎11/1/21,21:10:17,Bego,‎audio omitido,[]
1,‎12/1/21,13:14:04,Bego,‎audio omitido,[]
2,‎12/1/21,13:37:42,Ju,‎audio omitido,[]
3,12/1/21,14:22:35,Bego,Que guai amooooor la ofi no?,[]
4,12/1/21,14:22:50,Bego,Oiiii donali besis a la cami amore,[]
...,...,...,...,...,...
18066,28/4/23,15:09:53,Bego,tinc encara 10 min sino dpres,[]
18067,28/4/23,15:09:55,Bego,com vlguis,[]
18068,28/4/23,15:10:00,Bego,uina h vas casa leni,[]
18069,28/4/23,15:11:25,Ju,despres millor doncs amor,[]


## Emojis unique

In [8]:
# converts the row from a list of strings into a set of unique strings
df["Emojis"] = [''.join(set(row)) for row in df["Emojis"]]

# Split each string in the column into a list of characters
df['Emojis'] = df['Emojis'].apply(list)

# Explode the list of characters into separate rows
df = df.explode('Emojis')

# Convert each character back to a string
df['Emojis'] = df['Emojis'].apply(str)
df = df.reset_index(drop=True)
df## Emojis

,Date,Time,Name,Message,Emojis
0,‎11/1/21,21:10:17,Bego,‎audio omitido,nan
1,‎12/1/21,13:14:04,Bego,‎audio omitido,nan
2,‎12/1/21,13:37:42,Ju,‎audio omitido,nan
3,12/1/21,14:22:35,Bego,Que guai amooooor la ofi no?,nan
4,12/1/21,14:22:50,Bego,Oiiii donali besis a la cami amore,nan
...,...,...,...,...,...
20363,28/4/23,15:09:55,Bego,com vlguis,nan
20364,28/4/23,15:10:00,Bego,uina h vas casa leni,nan
20365,28/4/23,15:11:25,Ju,despres millor doncs amor,nan
20366,28/4/23,15:14:08,Bego,perfe🖤🌻,🖤


## Emojis clean

In [9]:
df['Emojis'] = df['Emojis'].replace('️', np.nan)
df['Emojis'] = df['Emojis'].replace('nan', np.nan)
df['Emojis'] = df['Emojis'].replace('♀', np.nan)
df['Emojis'] = df['Emojis'].replace('♂', np.nan)
df['Emojis'] = df['Emojis'].replace('🏽', np.nan)
df['Emojis'] = df['Emojis'].replace('🏼', np.nan)
df['Emojis'] = df['Emojis'].replace('🇦', np.nan)
df['Emojis'] = df['Emojis'].replace('🇩', np.nan)

df = df.dropna(subset=['Emojis'])


In [10]:
df

,Date,Time,Name,Message,Emojis
5,12/1/21,14:23:33,Bego,(I un per tu🌸),🌸
13,14/1/21,10:09:55,Bego,☀️☀️☀️☀️☀️☀️❤️,☀
15,14/1/21,10:09:55,Bego,☀️☀️☀️☀️☀️☀️❤️,☀
17,14/1/21,10:09:55,Bego,☀️☀️☀️☀️☀️☀️❤️,☀
19,14/1/21,10:09:55,Bego,☀️☀️☀️☀️☀️☀️❤️,☀
...,...,...,...,...,...
20316,26/4/23,23:28:53,Ju,marxaree amor ❤️❤️,❤
20342,27/4/23,14:21:38,Ju,❤️,❤
20353,28/4/23,12:26:34,Bego,et truco llavors???❤️,❤
20366,28/4/23,15:14:08,Bego,perfe🖤🌻,🖤


## Dates

In [11]:
# Convert Date into a datetime format
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Create new columns 
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')
df['Time'] = df['Time'].dt.time

df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Year'] = df['Date'].dt.year
df['Hour'] = df['Time'].apply(lambda x: x.hour)

In [12]:
sort_cols = ['Date', 'Day', 'Month', 'Year', 'Hour', 'Name', 'Emojis']
df = df.reindex(columns=sort_cols)
df

,Date,Day,Month,Year,Hour,Name,Emojis
5,2021-12-01,1,12,2021,14,Bego,🌸
13,2021-01-14,14,1,2021,10,Bego,☀
15,2021-01-14,14,1,2021,10,Bego,☀
17,2021-01-14,14,1,2021,10,Bego,☀
19,2021-01-14,14,1,2021,10,Bego,☀
...,...,...,...,...,...,...,...
20316,2023-04-26,26,4,2023,23,Ju,❤
20342,2023-04-27,27,4,2023,14,Ju,❤
20353,2023-04-28,28,4,2023,12,Bego,❤
20366,2023-04-28,28,4,2023,15,Bego,🖤


In [13]:
init_date = str(df["Date"].min().date())
init_date

'2021-01-04'

In [14]:
final_date = str(df["Date"].max().date())
final_date

'2023-12-04'

In [15]:
fecha_limite = pd.to_datetime("2023-04-23")
df = df.loc[df['Date'] < fecha_limite]
df = df.reset_index(drop=True)
df

,Date,Day,Month,Year,Hour,Name,Emojis
0,2021-12-01,1,12,2021,14,Bego,🌸
1,2021-01-14,14,1,2021,10,Bego,☀
2,2021-01-14,14,1,2021,10,Bego,☀
3,2021-01-14,14,1,2021,10,Bego,☀
4,2021-01-14,14,1,2021,10,Bego,☀
...,...,...,...,...,...,...,...
2670,2023-04-18,18,4,2023,21,Ju,😹
2671,2023-04-18,18,4,2023,21,Ju,😹
2672,2023-04-18,18,4,2023,21,Ju,😹
2673,2023-04-21,21,4,2023,18,Bego,❤


In [16]:
#df.to_csv("../data/chat_whatsapp.csv", index=False)

In [17]:
pd.isnull(df).sum()

Date      0
Day       0
Month     0
Year      0
Hour      0
Name      0
Emojis    0
dtype: int64

In [18]:
emojis_count = df['Emojis'].value_counts()
top_emojis = emojis_count[:15]
top_emojis

Emojis
❤    753
😂    532
😹    253
😻    113
💓     66
🖤     59
✨     57
🥺     56
🥲     46
💖     45
😍     43
💔     34
🥰     26
💛     25
🏻     22
Name: count, dtype: int64

# DRAFT

In [19]:
# Obtener valores únicos de cada fila en la columna "Emojis_2"
df['unique_emojis'] = df['Emojis_2'].apply(lambda x: list(set(x)))
# Convertir la columna de listas en una columna con un valor único en cada fila
df = df.explode('unique_emojis')
# Obtener los valores únicos de la columna "unique_emojis"
#unique_emojis = df['unique_emojis'].unique()
df = df.reset_index(drop=True)
df

KeyError: 'Emojis_2'

In [ ]:
#df.drop(df[df['unique_emojis'] == ''].index, inplace=True)
df

In [ ]:
pd.set_option ("display.max_rows", None) # Display all the columns
df.groupby("unique_emojis").agg("count")

In [ ]:
pd.DataFrame(df.groupby(["Name", "Year"])["unique_emojis"].count())

In [ ]:
#df['Emojis_2'] = df['Emojis'].apply(lambda x: [c for s in x for c in s])
#df.sample(10)

In [ ]:
#row = df["Emojis"]  # replace 0 with the index of the row you want to get the unique values from
#unique_values = list(set([c for s in row for c in s]))
#unique_values